<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 2.5. Magnetism &amp; magnetic field

Magnets and their magnetic field are found everywhere in daily life, e.g. loudspeakers, generators, computer memory, or in magnetic resonance imaging.
Magnets, or more specifically magnetic ore, were found more than thousand years ago in the greek region of Magnesia, hence, the name.
However, only in the 19th century the interdependency of magnetism and electricity has been discover.


## 2.5.1. Primer on magnets &amp; magnetic fields


Every magnet has a north pole and south pole.
Like electric charges, opposite poles attracts each other, while pole with the same orientation repel each other.
Unlike electric charges which can be monopoles, *all magnets are dipoles*.
That means, if you cut an bar magnet into two pieces you will end up with two smaller bar magnet, each with its own north-south-pole couple.
*No magnetic monopole*, i.e. a magnetic piece with only a single pole, has been observed.
Magnet produce a *magnetic field*.
Similar to electric fields, magnetic field lines are drawn according to these principles:
* the magnetic field line density is proportional to the strength of the magnetic field
* the direction of the underlying magnetic field is tangent to the field lines at any point
* the magnetic field lines always from north to south

However, due to the nature of magnetic dipoles, the *magnetic field lines*, there are some key differences compared to the electric field lines:
* magnetic field lines are always closed loops (they cannot begin or end like electric field lines at electric monopoles)
* hence, the magnetic field lines continue through the magnet itself

While we will discuss the concept in a later section more in detail, magnetic materials can be characterized how they behave when put into an external magnetic field.
Here a simplified view on the three different classes of magnetic materials:
*Diamagnetic* materials "push" the magnetic field outside, *paramagnetic* materials "pull" the magnetic field inside of them, and *ferromagnetic* materials have an even stronger "pulling" effect than paramagnetic materials.
From daily life you will know ferromagnetic materials such as iron (in Latin "ferrum")  but also other materials such as nickel and cobalt are ferromagnetic.

Compasses are another typical example of magnetic fields in daily life.
In essence, they are suspended bar magnets used to detect the Earth's magnetic field.
The Earth's magnetic field itself can be reasonably well be approximated as a (rather big) bar magnet.
By definition, the pole of a compass (or any other magnet) is called *north* if it points towards the Earths geographic north pole (technically that would be the Earth's magnetic south pole but even that is sometimes referred to as the geomagnetic north pole). 

**Disclaimer**:
We will use $\vec{\bf{B}}$ to represent the magnetic field.
Without going to much into detail, there are two entities, i.e. $\vec{\bf{B}}$ and $\vec{\bf{H}}$ to describe the magnetic field.
$\vec{\bf{B}}$ is actually the magnetic flux density or (as we see later) simply called induction.
$\vec{\bf{H}}$ is the actual magnetic field strength. 
While $\vec{\bf{B}}$ is expressed in the units *Tesla* [T], equivalent to [N / (A m)], the unit of $\vec{\bf{H}}$ is [A/m].
Both can be related to each other, i.e. in vacuum we know $\vec{\bf{H}} = \frac{1}{\mu_0} \vec{\bf{B}} - \vec{\bf{M}}$ with $\mu_0$ and $\vec{\bf{M}}$ being the permeability of free space and magnetization, respectively.

Keeping things simple, we will use the $\vec{\bf{B}}$ field exclusively in this script when referring to the magnetic field.


In [1]:
# generated with AI, tweaked by me, and adapted from https://scipython.com/blog/visualizing-the-earths-magnetic-field/

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from ipywidgets import interact, FloatSlider, Checkbox
from scipy.constants import mu_0, pi

# Function to calculate the magnetic field of a dipole at position r0
def calc_b_field_per_dipole(m, r0, x, y):
    r_vec = np.array([x - r0[0], y - r0[1]])
    r = np.hypot(r_vec[0], r_vec[1])
    r_hat = r_vec / (r + 1e-9)  # Avoid division by zero
    
    # Compute dot product element-wise
    m_dot_r_hat = m[0] * r_hat[0] + m[1] * r_hat[1]
    
    # Compute B-field components using broadcasting
    B_magnitude = (mu_0 / (4 * pi)) * (3 * m_dot_r_hat * r_hat - np.expand_dims(m, axis=(1,2))) / (r**3 + 1e-9)
    
    return B_magnitude[0], B_magnitude[1]

# Width of the bar magnets
MAGNET_WIDTH = 0.3

# Grid of x, y points
nx, ny = 64, 64
EXTENT = 3  # Space around magnets
x = np.linspace(-EXTENT, EXTENT, nx)
y = np.linspace(-EXTENT, EXTENT, ny)
X, Y = np.meshgrid(x, y)

def plot_magnetic_field(distance, magnet_length, flip_left):
    # Number of dipoles to approximate bar magnet
    num_dipoles = 32
    dipole_spacing = magnet_length / num_dipoles
    
    # Define dipole moments and positions for two bar magnets
    m1_list = [[0, -1]] * num_dipoles if flip_left else [[0, 1]] * num_dipoles
    m2_list = [[0, -1]] * num_dipoles
    
    pos1_list = [[-distance / 2, -magnet_length / 2 + i * dipole_spacing] for i in range(num_dipoles)]
    pos2_list = [[distance / 2, -magnet_length / 2 + i * dipole_spacing] for i in range(num_dipoles)]
    
    # Compute the magnetic field
    Bx, By = np.zeros((ny, nx)), np.zeros((ny, nx))
    for m, pos in zip(m1_list + m2_list, pos1_list + pos2_list):
        bx, by = calc_b_field_per_dipole(m, pos, X, Y)
        Bx += bx
        By += by
    
    # Plot
    fig, ax = plt.subplots()
    color = 2 * np.log1p(np.hypot(Bx, By))
    strm = ax.streamplot(x, y, Bx, By, color=color, linewidth=1, cmap=plt.cm.inferno,
                          density=2, arrowstyle='->', arrowsize=1)
    
    # Mask field lines inside magnets by drawing the rectangles on top
    for pos, flip in [(-distance / 2, flip_left), (distance / 2, False)]:
        if flip:
            ax.add_patch(Rectangle((pos - MAGNET_WIDTH / 2, 0), MAGNET_WIDTH, magnet_length / 2, color='b', alpha=1, zorder=10))
            ax.add_patch(Rectangle((pos - MAGNET_WIDTH / 2, -magnet_length / 2), MAGNET_WIDTH, magnet_length / 2, color='r', alpha=1, zorder=10))
        else:
            ax.add_patch(Rectangle((pos - MAGNET_WIDTH / 2, -magnet_length / 2), MAGNET_WIDTH, magnet_length / 2, color='b', alpha=1, zorder=10))
            ax.add_patch(Rectangle((pos - MAGNET_WIDTH / 2, 0), MAGNET_WIDTH, magnet_length / 2, color='r', alpha=1, zorder=10))
    
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_xlim(-EXTENT, EXTENT)
    ax.set_ylim(-EXTENT, EXTENT)
    ax.set_aspect('equal')
    plt.show()

interact(plot_magnetic_field, 
         distance=FloatSlider(value=1.5, min=MAGNET_WIDTH, max=EXTENT, step=0.1, description='Separation'),
         magnet_length=FloatSlider(value=1.2, min=MAGNET_WIDTH, max=EXTENT, step=0.1, description='Magnet Length'),
         flip_left=Checkbox(value=False, description='Flip Left Magnet'))


interactive(children=(FloatSlider(value=1.5, description='Separation', max=3.0, min=0.3), FloatSlider(value=1.…

<function __main__.plot_magnetic_field(distance, magnet_length, flip_left)>

## 2.5.2 Forces &amp; effects of the magnetic field

In this section, we will briefly introduce how magnetic field's &amp; electric charges interact and what forces come into play.

### Electric currents produce magnetic fields
While a static electric charge and magnet show no interaction, Hans Christian &Oslash;rsted (1777-1851) found that the needle of a compass would deflect if put near a wire running current.
That was a major breakthrough in connecting electricity and magnetism.
&Oslash;rsted could conclude, a *electric current produces a magnetic field*. 
Further, the magnetic field created by the current is not uniform in direction and magnitude.
By placing the compass at different positions, one can see that the field forms in circles around the wire (compass needle aligns tangent to field lines).
With increasing distance $r$ from the wire, the compass needle is less deflected.
Further, the deflection of the needle scales with the current $I$ running through the wire.
Therefore, the magnitude of the magnetic field is proportional to:
$$ B \propto \frac{I}{r} $$

In general, the magnitude of magnetic field $B$ produced by a straight current-running wire is:
$$B = \frac{\mu_0}{2 \pi} \frac{I}{r} $$
with the proportionality constant $\frac{\mu_0}{2 \pi}$ .
The constant $\mu_0$ is the *permeability of free space* and defined as $4 \pi \times 10^-7$ T m /A.


### Magnetic fields exert a force on electric currents
If a electric current is exerting a force on a magnet and motivated by Newton's third law, a magnetic field might also exert a force on a wire running electric current.
&Oslash;rsted conducted a simple but elegant experiment.
He placed a straight wire in homogenous magnetic field (e.g. reasonably well approximated by a strong horseshoe magnet).
If he ran a current through the wire, the wire was bend.
While, the exact bending direction depended on the direction the current was running, i.e. the polarity of the voltage source, the bending occurred always *perpendicular* the the magnetic field.

The strength of the force on the wire can be studied experimentally to derive a equation.
Let's start easy by keeping the wire perpendicular to the magnetic field. 
We would observe that the force $F$ is proportional to the amplitude of magnetic field $\vec{\bf{B}}$.
Further, it scales with the length $l$ of the wire inside the field and the current $I$ running through the wire.
If we would keep $l$, $I$, and $B$ constant, but change the orientation of the wire within the field, we would observe that the force is zero if the wire and magnetic field are parallel, and maximal, if the wire and magnetic field are perpendicular.
Therefore, for a homogenous magnetic field $B$ and $\theta$ the angle between $B$ and $l$ we obtain:
$$ F = Il B \sin \theta$$

Extending this to the vector form, yields
$$ \vec{\bf{F}} = I \vec{\bf{l}} \times \vec{\bf{B}} $$

and, if seen for infinitesimal small wire elements $d \vec{\bf{l}}$ and non-uniform magnetic field, we obtain
$$ d \vec{\bf{F}} = I d\vec{\bf{l}} \times \vec{\bf{B}} $$

Note that these equation use the conventional current, i.e. running from positive to negative, and not the true physical direction of freely moving electrons.

### Notes on magnetic fields in diagrams and the right-hand rules
Given the 3D nature of the force-current-magnetic field interaction and our typical 2D diagrams, by convention the magnetic field if often draws "perpendicular to the piece of paper".
If the field lines would "go into the paper" we draw them as $\odot$ and if the "come out of the paper" we draw them as $\times$ which resembles the tip and end of an arrow, respectively.

To quickly predict the above mentioned interactions, simply remember the following rules:
* *Right-Hand Rule 1* (RHR-1): 
    * thumb pointing along the direction of the current $I$ in a wire
    * wrapped fingers "around"
    * fingers curl in the direction the magnetic field $\vec{\bf{B}}$
* *Right-Hand Rule 2* (RHR-2): 
    * thumb pointing along the direction of the current $I$ 
    * index finger points perpendicular to your thumb along the direction of the magnetic field $\vec{\bf{B}}
    * middle finger perpendicular to thumb &amp; index finger, points in the direction of the force


### Force between two parallel wires
If currents produce magnetic fields and magnetic fields exert force on wires running currents, putting two straight, current-running wires parallel to each other, is an interesting (thought) experiment.
From intuition, we expect that both will exert force on the other.

Let's define the distance between the parallel wires as $d$ and have each wire running its direct current $I_1$ and $I_2$, respectively.
The magnitude of the field produced by wire $1$ at the position of wire $2$, i.e. at distance $d$, would be:
$$B_1 =  \frac{\mu_0}{2 \pi} \frac{I_1}{d}$$

The force $F_{21}$ exerted by $B_1$ on the wire $2$ will be:
$$ F_{21} = I_2 l_2 B_1$$

Note that the current $I_2$ also produces a magnetic field $B_2$, but this does not exert a force on wire $2$ itself but only on wire $1$.
We combine both equation to describe the force $F_{21}$ further:
$$ F_{21} = \frac{\mu_0}{2 \pi} \frac{I_1 I_2}{d} l_2$$

Using RHR-2, we can see that if the currents $I_1$ and $I_2$ are in the same direction, the forces cause the wires to attract each other, and, if the current are in opposite direction, the forces cause the wires to repel each other.

### Torque on a current loop &amp; the magnetic dipole moment
Instead of straight wires, lets consider a flat, rectangular loop parallel to a magnetic field.
We orient the wire inside the field such as that the portion with the connections has the current running parallel to the field.
Hence, no force is exerted as the angle between current $I$ and and magnetic field $\vec{\bf{B}}$ is zero.
However, there are two parts of the rectangular loop perpendicular to the magnetic field.
Therefore, there a force will be exerted.
Lets assume the rectangular loop can rotate around its symmetry axis and that the length of the loop parallel to the field is $b$ and the length perpendicular to the field is $a$.
According to the RHR-2, the two sides which have current running perpendicular to the field will be associated with the forces $\vec{\bf{F}}_1$ and $\vec{\bf{F}}_2$, both with the same magnitude $| \vec{\bf{F}} | = I a B$ but opposite direction (current runs in opposite direction in both ports of the loop). 
As the coil can rotate, $\vec{\bf{F}}_1$ and $\vec{\bf{F}}_2$ result in a net torque $\tau$. 
As the lever arm is $\frac{b}{2}$, the magnitude of the net torque is:
$$ \tau = F_1 \frac{b}{2} + F_2 \frac{b}{2} $$
$$ \tau = I a B \frac{b}{2} + I a B \frac{b}{2} = I a b B $$
$$ \tau = I A B  $$
with the area of the rectangular coil $A = a b$.

Assuming we have multiple coil loops, $N$, hence an effective current of $N I$, and oriented at an angle $\theta$ w.r.t. to the magnetic field, we can state:
$$ \tau = N I A B \sin \theta $$

This idea of a suspended coil loop inside a magnetic field is the basis for many devices such as motors and analog multimeters. 
Beyond rectangular coils, this concept is true for any flat coil with arbitrary shape.
In line with the electric dipole, we can introduce the concept of a *magnetic dipole moment*.
We define the vector $\vec{\bf{A}}$ as a vector perpendicular to the area of the coil.
Then, the magnetic dipole moment $\vec{\bf{\mu}}$ is defined as:
$$  \vec{\bf{\mu}} = N I \vec{\bf{A}} $$
For the torque, we obtain:
$$ \vec{\bf{\tau}} = N I \vec{\bf{A}} \times \vec{\bf{B}} $$
$$ \vec{\bf{\tau}} = \vec{\bf{mu}} \times \vec{\bf{B}} $$

Similar to the potential energy of an electric dipole, the potential energy of a magnetic dipole is:
$$ U = - \vec{\bf{p}} \vec{\bf{B}}



In [ ]:
# todo simulation

## 2.5.3. Laws of magnetostatics

We have already discussed the magnetic field due to a straight, current-running wire.
However, how to generalize to arbitrary configurations?
This will be the subject of this section and deals on how to describe magnetic fields due to time-constant/stationary currents, i.e. discuss laws of magnetostatics.

### Ampère's law
Inspired by &Oslash;rsted's observation, André Marie Ampère (1775-1836) proposed a general relation between arbitrarily shaped, current-running wires and their associated magnetic field.

Let's consider an arbitrarily-shaped, closed path around a current.
We call the enclosed current $I_{enc}$ and divide the path into short segments $\Delta l$.
For these short segments, we consider only the component of the magnetic field parallel $B_{\parallel}$ to the segment. 
By summing up all segmented with their respective $B_{\parallel}$, we get an discrete approximation of Ampère'ss law:

$$ \sum B_{\parallel} \Delta l = \mu_0 I_{enc} $$

Note that the sum is equal to the enclosed current $I_{enc}$ times the permeability of free space $\mu_0$.

In the limit, i.e. the segments become infinitesimal short, we obtain *Ampère's law* by taking the line integral $\oint$ around the enclosed current:
$$ \oint \vec{\bf{B}} \cdot d \vec{\bf{l}} = \mu_0 I_{enc} $$

As the segments become infinitesimal short, the vector dot product with the magnetic field $\vec{\bf{B}}$ ensure we consider only the parallel component.
The beauty of Ampère's law is that is also provides valid estimates of $\vec{\bf{B}}$ if there are currents outside the enclosed path (same is true for Gauss's and charges outside the enclosed surface).


### Biot-Savart law
While Ampère's law is our most general description, it relies on symmetry of the problem at hand to be analytically solved.
The two scientist Jean Baptiste Biot (1774-1862) and Felix Savart (1791-1841) developed an alternative approach to determine the magnetic field due to a current.
Similar to Coulomb's law in differential form, $ dE = \frac{1}{4 \pi \epsilon_0} \frac{dQ}{r^2} $, they proposed the following:
An infinitesimal short current element $d \vec{\bf{l}}$ will produce the magnetic field $d \vec{\bf{B}}$ at a point in space characterized by $\vec{\bf{r}}$:
$$ d \vec{\bf{B}} = \frac{\mu_0 I}{4 \pi} \frac{d \vec{\bf{l}} \times  \hat{\bf{r}}}{r^2} $$

with $\hat{\bf{r}} = \frac{\vec{\bf{r}}}{r}$ as the unit displacement vector.
By integrating the equation above, we obtain the *Biot-Savart law*;

$$ \vec{\bf{B}} = \int d \vec{\bf{B}} = \frac{\mu_0 I}{4 \pi} \int \frac{d \vec{\bf{l}} \times  \hat{\bf{r}}}{r^2} $$

Note that Biot-Savart law, only computes the magnetic field due to the current inside the wire segments $d \vec{\bf{l}}$ (unlike Ampère's law).
Hence, for a truthful description, all current in a scenario need to be considered and not just the enclosed ones.


<!--- todo: magnetic dipole field-->



## 2.5.4. Individual charges moving through magnetic fields

To understand what happens to a single electro or proton when moving through a field (without being inside a wire), we can combine our knowledge of what happens to a current-running wire inside a magnetic field with the definition of current.
Since electric current $I$ was defined as the charge per unit time, we can adapt this equation to the number of individual charges $N$ multiplied with the charge of the particles $q$ per unit time $t$:
$$ I = \frac{N q}{t} $$
Further, the charged particles travel the distance $l$ depending on their speed $v$:
$$ \vec{\bf{l}} = t \vec{\bf{v}} $$

Therefore, we adapt the equation of a wires inside a magnetic field to be applicable to a single charge ($N=1$) by:
$$ \vec{\bf{F}} = I \vec{\bf{l}} \times \vec{\bf{B}} = \frac{N q}{t} t \vec{\bf{v}} \vec{\bf{B}}$$
$$ \vec{\bf{F}} = q \vec{\bf{v}} \times \vec{\bf{B}}  $$

As before, if $\vec{\bf{B}}$ is uniform, the equation can be simplified to 
$$ F = q v B \sin \theta $$
with $theta$ as the angle between the magnetic field and the direction the charge in moving.

Note that you can apply the RHR-2 for positively charged particles with your thumb pointing in the direction the particle is moving.
For negatively charged particles, i.e. electrons, you have to use the left hand, but otherwise the logic is the same.

Assuming a charged particle moving perpendicular to an uniform magnetic field, there will be a forces exerted on the particles.
The force will deflect the particle and, as the force is always perpendicular to the direction of movement, causes the particle to move on a circular path (if the magnetic field is "large enough") with a centripetal acceleration magnitude of $ a = \frac{v^2}{r}$ (see mechanics lectures).
For a circular trajectory, the centripetal force and the force due to the magnetic field must be equal
$$ \sum F = m a$$
$$ q v B = m \frac{v^2}{r} $$

Therefore, the radius is:
$$ r = \frac{m v^2}{q v B}  = \frac{m v}{q B} $$


### Determining the electron's charge-to-mass-ratio
If an electron with charge $e$ and mass $m_e$ is first accelerated by an anode-cathode pair and subsequently entering perpendicular into a uniform magnetic field $B$, we can write:
$$ e v B = m_e \frac{v^2}{r} $$
$$ \frac{e}{m_e} = \frac{v}{B r} $$
If we use a discharge tube, we can measure the radius $r$.
While we can measure experimentally or estimate analytically $B$, we still need to determine $v$.
To that end, we will use two plates to generate an electric field $E$ perpendicular to the magnetic field $B$.
This combination of $E$- and $B$-field is a so-called *velocity selector*.
If the deflection of the particle due to both field is the same, i.e. the force due to the electric field and due to the magnetic field are equal, the electron moves in a straight line through the $E$-$B$-field combination:
$$ F_{ele} = F_{mag} $$
$$ e E = e v B $$
Therefore, the velocity can be expressed as the ratio of both (non-zero) fields:
$$ v = \frac{E}{B} $$

Using this relation, give us our final equation to estimate the electron's charge-to-mass-ratio:
$$ \frac{e}{m_e} = \frac{E}{B^2 r} $$

The numerical value of $\frac{e}{m_e}$ is 1.76 $\times$ C/kg.


<!--- todo: image cathode ray tube --> 

### Mass spectrometer
The concept of particles insides fields and velocity selectors can be extended to generate mass spectrometers.
These are devices to measure the masses of atoms.
Conceptional, an anode-cathode pair emits (by heating) and accelerates ions (mass $m$ and charge $q$). 
These ions enter a velocity selector, i.e. a perpendicular $E$-$B$-field combination. 
Behind the velocity selector is a piece of metal with a small slit. 
Only ions with a straight trajectory, i.e. with a velocity of $v = \frac{E}{B} $ will pass through the velocity-slit-combination.
Afterwards, the ions with velocity $v$ enter perpendicularly into another uniform magnetic field $B_d$ to deflect the ions.
This magnetic field will force the ions on a semicircular path before they hit and detector. 
The detector is at a constant distance of $2r$ from the slit and the magnetic field $B_d$ adjusted until the ions are deflected in such a way that they are detected by the detector.
As shown above, the centripetal force and force due to the magnetic field can be related by $q v B_d = m \frac{v^2}{r}$
To summarize:
* the ion velocity is selected according to $v = \frac{E}{B} $
* the deflection due to the seconds magnetic field is found via $q v B_d = m \frac{v^2}{r}$

As we want to estimate the mass of ions, we combine and rearrange the equations as follows:
$$ q v B_d = m \frac{v^2}{r} $$
$$ q B_d = m \frac{v}{r} $$
$$ q B_d = m \frac{\frac{E}{B}}{r} $$
$$ q B_d = m \frac{E}{B r}$$
$$ m = \frac{q B_d B r}{E} $$ 

Nowadays, mass spectrometers are used not only in physics but many other field such as chemistry and medicine.
Further, mass spectrometers are great to illustrate the existence of *isotopes*. 
For example, mass spectrometry of neon would yield detection radii corresponding to two different atomic masses.
This difference in atomic mass is due to a different number of neutrons. 
Hence, isotopes of an element have the same atomic number (number of protons) but different atomic mass.


### Lorentz equation

The effects of forces on a particle due to electric and magnetic fields can be generalized by the *Lorentz equation*:
$$ \vec{\bf{F}} = \vec{\bf{F}}_{ele} + \vec{\bf{F}}_{mag}$$
$$ \vec{\bf{F}} = q \vec{\bf{E}} + q \vec{\bf{v}} \times \vec{\bf{B}} $$
$$ \vec{\bf{F}} = q (\vec{\bf{E}} + \vec{\bf{v}} \times \vec{\bf{B}}) $$


### Hall effect

So far we have looked at charged particles in vacuum (or air), but a magnetic field does also influence the electrons running inside a conductor, i.e. a wire.
We know that the force $\vec{\bf{F}}_B$ exerted by the $\vec{\bf{B}}$ field on a electron $e$ be:
$$ \vec{\bf{F}}_B = e \vec{\bf{v}}_d \times \vec{\bf{B}}$$
with $\vec{\bf{V}}_d$ being the drift velocity of the electron.

Therefore, the electron will be deflected towards one side of the conducting wire.
The resulting potential difference generates an electric field $\vec{\bf{E}}_H$ across the cross-section of the wire, which we call the *Hall field*.
The hall field itself causes a force with the same magnitude but opposite direction to the magnetic force:
$$ e E_H = e v_d B $$
$$ E_H = v_d B $$

The resulting *Hall voltage* $V_H$ in the presence of uniform, perpendicular fields and thin, but long conducting wire is:
$$ U_H = E_H d = v_d B d $$

The intriguing part about the *Hall effect* is that it can differentiate between positive and negative charges.
Assuming the same magnitude of charge, negative and positive charges would be deflected by the same amount, but in different directions.
Hence, the hall voltage across the cross-section of the conductor will have opposite sign.
Further, *Hall probes* leverage the fact that the voltage $V_H$ is proportional to $B$ to measure magnetic fields.
Last but not least, the drift velocity of charged particles can be determined with Hall probes.


<!--- todo: images-->

<!--- todo: Magnetic field due to a single moving charge-->

<!--- todo: applications i.e. galvanometers, motors, speakers-->

In [ ]:
# todo simulation

## 2.5.5 Magnetic materials
As we learned at the beginning of this chapter, currents and magnets, i.e. magnetic materials can produce magnetic fields.
Here we will go into a bit more detail on magnetic materials and their (magnetic) susceptibility.

### Microscopic view of magnetism

At the microscopic scale, ferromagnetic materials, e.g. iron, can be divided into small region, the so-called *domains*.
Each domain is associated with is own north and south pole and, therefore, resembles like a tiny bar magnet.
If not magnetized, the fields of the individual domains cancel each other.
By using an external magnetic field, we can enforce a preferential direction of the domains and, therefore, magnetize the material.
We can reverse this effect by brute force, i.e. hitting the magnet, or by heating, i.e. reaching the *Curie temperature*, which causes the domains to align randomly.

Looking at the atomically level, we realize that particles themselves are associated with the fundamental property of a *spin*.
This spin gives rise to a magnetic field and today all magnetic fields are believed to be essentially the consequence of electric currents at the atomic scale.


### Magnetic permeability &amp; susceptibility

Besides the permeability of free space $\mu_0$, there is the material-specific magnetic *permeability* $\mu$.
The *relative permeability* $K_m$ is defined as:
$$ K_m = \frac{\mu}{\mu_0}$$
and the *magnetic susceptibility* $\chi$ (Greek small-case chi) is directly related to this ratio as
$$ \chi = K_m - 1 $$

Therefore, if the permeability of a material is less than the permeability of free space $\mu_0$, its susceptibility $\chi$ is negative.
In essence, the susceptibility quantifies how magnetized a material becomes when put into an external magnetic field.
Materials with $\chi < 0$ are called *diamagnetic*.
Materials with $\chi > 0$ are called *paramagnetic*.
Interestingly, oxygenated blood is diamagnetic, and deoxygenated blood is paramagnetic, which is the basis for blood oxygenation level dependent (BOLD) functional magnetic resonance imaging (as story for another time). 
By the way, *ferromagnetic* materials are "extreme" cases with $\chi \gg 1$

### Hysteresis

Hysteresis, in the context of a magnetizable material, describes how the material's own magnetic field $\vec{\bf{B}}$ responds to changes in an external magnetic field $\vec{\bf{B}}_0$.


Imagine a piece of magnetizable iron inside a coil.
We can control the field $\vec{\bf{B}}_0$ of the coil by adjusting the current (see next chapter for details).
If we start with no external magnetic field and then gradually increase it by increasing the current in the coil, the iron's own magnetic field $\vec{\bf{B}}$ will also increase in response to this external field until is *saturates*.

Now, if we start to decrease the external magnetic field $\vec{\bf{B}}_0$ again, the iron's magnetic field $\vec{\bf{B}}$ will also decrease.  
However, and this is key for hysteresis, it does not decrease in the exact same way it increased. 
Specifically, for a given value of the external magnetic field on the way down, the iron's magnetic field will be stronger than it was for the same external field on the way up.

Even if you reduce the external magnetic field completely back to zero, the iron will still have its own magnetic field $\vec{\bf{B}}$ remaining.  
To completely remove the magnetic field B from the iron and bring it back to zero, you would need to apply an external magnetic field in the opposite direction.

This lagging effect, where the material's magnetic field B depends not just on the current external magnetic field but also on the past history of the external magnetic field, is called hysteresis.  
It means the magnetic state of the iron is not solely determined by the instantaneous external field but also by its previous magnetic states.

Permanent magnets are characterized by a broad hysteresis loop, while electromagnets (see next chapter) are characterized by more sallow curves.


<!--- todo: images-->

